In [20]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [16]:
processed_data_dir = '..\\data\\processed\\'

In [5]:
file_path = '..\\data\\raw\\Churn_Modelling.csv'

In [6]:
try:
    df = pd.read_csv(file_path)
    print(f"Данные успешно загружены. Размер: {df.shape}")
except FileNotFoundError:
    print(f"Ошибка: Файл не найден по пути '{file_path}'. Проверьте путь.")

Данные успешно загружены. Размер: (10000, 14)


In [7]:
columns_to_drop_initial = ['RowNumber', 'CustomerId', 'Surname']
df_processed = df.drop(columns=columns_to_drop_initial)

In [8]:
columns_to_drop_useless = ['HasCrCard']
if columns_to_drop_useless[0] in df_processed.columns:
    df_processed = df_processed.drop(columns=columns_to_drop_useless)
    print(f"Удалены бесполезные колонки: {columns_to_drop_useless}")

Удалены бесполезные колонки: ['HasCrCard']


In [9]:
print("\n--- Создание новых признаков ---")

df_processed['IsBalanceZero'] = (df_processed['Balance'] == 0).astype(int)
print("Создан признак 'IsBalanceZero'.")

df_processed['HighNumProducts'] = (df_processed['NumOfProducts'] >= 3).astype(int)
print("Создан признак 'HighNumProducts'.")

df_processed['BalancePerProduct'] = np.where(
    df_processed['NumOfProducts'] > 0,
    df_processed['Balance'] / df_processed['NumOfProducts'],
    0
)
print("Создан признак 'BalancePerProduct'.")

df_processed['TenurePerAge'] = np.where(
    df_processed['Age'] > 0,
    df_processed['Tenure'] / df_processed['Age'],
    0
)
print("Создан признак 'TenurePerAge'.")


--- Создание новых признаков ---
Создан признак 'IsBalanceZero'.
Создан признак 'HighNumProducts'.
Создан признак 'BalancePerProduct'.
Создан признак 'TenurePerAge'.


In [10]:
print("\nПризнаки после инжиниринга:")
print(df_processed.head())
print("Новый размер:", df_processed.shape)


Признаки после инжиниринга:
   CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0          619    France  Female   42       2       0.00              1   
1          608     Spain  Female   41       1   83807.86              1   
2          502    France  Female   42       8  159660.80              3   
3          699    France  Female   39       1       0.00              2   
4          850     Spain  Female   43       2  125510.82              1   

   IsActiveMember  EstimatedSalary  Exited  IsBalanceZero  HighNumProducts  \
0               1        101348.88       1              1                0   
1               1        112542.58       0              0                0   
2               0        113931.57       1              0                1   
3               0         93826.63       0              1                0   
4               1         79084.10       0              0                0   

   BalancePerProduct  TenurePerAge  
0           0.

In [11]:
print("\n--- Разделение данных на обучающую и тестовую выборки ---")
X = df_processed.drop('Exited', axis=1)
y = df_processed['Exited']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"Размер X_train: {X_train.shape}")
print(f"Размер X_test: {X_test.shape}")
print(f"Доля оттока в y_train: {y_train.mean():.3f}")
print(f"Доля оттока в y_test: {y_test.mean():.3f}")


--- Разделение данных на обучающую и тестовую выборки ---
Размер X_train: (8000, 13)
Размер X_test: (2000, 13)
Доля оттока в y_train: 0.204
Доля оттока в y_test: 0.203


In [12]:
print("\n--- Определение типов признаков и создание препроцессора ---")

X_train['Gender'] = X_train['Gender'].map({'Male': 1, 'Female': 0})
X_test['Gender'] = X_test['Gender'].map({'Male': 1, 'Female': 0})
print("Признак 'Gender' преобразован в числовой (0/1).")

categorical_features_ohe = ['Geography']
numerical_features = X_train.select_dtypes(include=np.number).columns.tolist()

numerical_features = [col for col in numerical_features if col not in categorical_features_ohe]

print(f"\nКатегориальные признаки для OHE: {categorical_features_ohe}")
print(f"Числовые признаки для масштабирования: {numerical_features}")


--- Определение типов признаков и создание препроцессора ---
Признак 'Gender' преобразован в числовой (0/1).

Категориальные признаки для OHE: ['Geography']
Числовые признаки для масштабирования: ['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'IsActiveMember', 'EstimatedSalary', 'IsBalanceZero', 'HighNumProducts', 'BalancePerProduct', 'TenurePerAge']


In [13]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False), categorical_features_ohe)
    ],
    remainder='passthrough'
)

In [14]:
print("\n--- Применение препроцессора к данным ---")

preprocessor.fit(X_train)

X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

num_feature_names = numerical_features
ohe_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features_ohe).tolist()

final_feature_names = num_feature_names + ohe_feature_names

print(f"Размер обработанного X_train: {X_train_processed.shape}")
print(f"Размер обработанного X_test: {X_test_processed.shape}")
print(f"Количество признаков после обработки: {len(final_feature_names)}")
print("Имена признаков после обработки:", final_feature_names)


--- Применение препроцессора к данным ---
Размер обработанного X_train: (8000, 14)
Размер обработанного X_test: (2000, 14)
Количество признаков после обработки: 14
Имена признаков после обработки: ['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'IsActiveMember', 'EstimatedSalary', 'IsBalanceZero', 'HighNumProducts', 'BalancePerProduct', 'TenurePerAge', 'Geography_Germany', 'Geography_Spain']


In [15]:
X_train_final = pd.DataFrame(X_train_processed, columns=final_feature_names, index=X_train.index)
X_test_final = pd.DataFrame(X_test_processed, columns=final_feature_names, index=X_test.index)

print("\n--- Первые 5 строк обработанной обучающей выборки (X_train_final) ---")
print(X_train_final.head())


--- Первые 5 строк обработанной обучающей выборки (X_train_final) ---
      CreditScore    Gender       Age    Tenure   Balance  NumOfProducts  \
2151     1.058568  0.907507  1.715086  0.684723 -1.226059      -0.910256   
8392     0.913626  0.907507 -0.659935 -0.696202  0.413288      -0.910256   
5006     1.079274 -1.101919 -0.184931 -1.731895  0.601687       0.808830   
4117    -0.929207  0.907507 -0.184931 -0.005739 -1.226059       0.808830   
7182     0.427035  0.907507  0.955079  0.339492  0.548318       0.808830   

      IsActiveMember  EstimatedSalary  IsBalanceZero  HighNumProducts  \
2151       -1.030206         1.042084       1.329364        -0.182185   
8392       -1.030206        -0.623556      -0.752239        -0.182185   
5006        0.970680         0.308128      -0.752239        -0.182185   
4117       -1.030206        -0.290199       1.329364        -0.182185   
7182        0.970680         0.135042      -0.752239        -0.182185   

      BalancePerProduct  TenurePe

In [21]:
print("\n--- Сохранение обработанных данных ---")

x_train_path = os.path.join(processed_data_dir, 'X_train_processed.csv')
x_test_path = os.path.join(processed_data_dir, 'X_test_processed.csv')
y_train_path = os.path.join(processed_data_dir, 'y_train.csv')
y_test_path = os.path.join(processed_data_dir, 'y_test.csv')

try:
    X_train_final.to_csv(x_train_path, index=False)
    print(f"Обучающие признаки сохранены в: '{x_train_path}'")

    X_test_final.to_csv(x_test_path, index=False)
    print(f"Тестовые признаки сохранены в: '{x_test_path}'")

    y_train.to_csv(y_train_path, index=False, header=True)
    print(f"Обучающая целевая переменная сохранена в: '{y_train_path}'")

    y_test.to_csv(y_test_path, index=False, header=True)
    print(f"Тестовая целевая переменная сохранена в: '{y_test_path}'")

    print("\n--- Все обработанные данные успешно сохранены. ---")

except Exception as e:
    print(f"\n--- Ошибка при сохранении данных: {e} ---")


--- Сохранение обработанных данных ---
Обучающие признаки сохранены в: '..\data\processed\X_train_processed.csv'
Тестовые признаки сохранены в: '..\data\processed\X_test_processed.csv'
Обучающая целевая переменная сохранена в: '..\data\processed\y_train.csv'
Тестовая целевая переменная сохранена в: '..\data\processed\y_test.csv'

--- Все обработанные данные успешно сохранены. ---
